<a href="https://colab.research.google.com/github/MeidanGR/GooglePlayStore_Preprocessing_Python/blob/main/GooglePlayStore_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Play Store Data Preprocessing**

---


This data preprocessing progress has been established for an [academic project](https://sites.google.com/view/biplaystore/home?authuser=0) of a BI course. The data will be analyzed within the project.

Major thanks to **Gautham Prakash** for publishing & maintenance this dataset, which you can find at https://www.kaggle.com/gauthamp10/google-playstore-apps


# **GOOGLE AUTH & LIBRARIES**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

# **CSV LOAD**


In [3]:
#Loading csv
df = pd.read_csv('/content/drive/My Drive/Google-Playstore.csv')

print("Dataframe original shape is", np.shape(df), end='\n\n')
print(df.info())
df.sample(2)

Dataframe original shape is (603047, 23)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603047 entries, 0 to 603046
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   App Name           603046 non-null  object 
 1   App Id             603047 non-null  object 
 2   Category           603047 non-null  object 
 3   Rating             599287 non-null  float64
 4   Rating Count       599287 non-null  float64
 5   Installs           602875 non-null  object 
 6   Minimum Installs   602875 non-null  float64
 7   Maximum Installs   603047 non-null  int64  
 8   Free               603047 non-null  bool   
 9   Price              603047 non-null  float64
 10  Currency           602866 non-null  object 
 11  Size               603047 non-null  object 
 12  Minimum Android    602556 non-null  object 
 13  Developer Id       603045 non-null  object 
 14  Developer Website  423783 non-null  object 
 15  Developer

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Currency,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
60802,Attitude Shayari in Hindi,com.best.djattitudeshayari,Entertainment,4.4,84.0,"10,000+",10000.0,41546,True,0.0,USD,12M,4.1 and up,Android Rock App,NaN,ambanidj90@gmail.com,"Dec 2, 2016",27/02/2019 15:26,Everyone,http://todaydealscoupon.com/Apps/aatitudestatu...,True,False,False
568662,USA Hockey Mobile Coach,com.usahockey.android.usahockey,Sports,4.8,1031.0,"50,000+",50000.0,79979,True,0.0,USD,9.5M,4.4 and up,"USA Hockey, Inc.",http://www.usahockey.com,usahmobilecoachassistance@gmail.com,"Oct 18, 2012",12/09/2018 21:19,Everyone,https://www.usahockey.com/termsofuse,False,True,False


# **DATA FILTERING**
## Agenda: 
Data reduction down to ~100K rows, and only relavent columns for a decent dataframe volume, shape, & a faster run in Google Data Studio.

## Guidelines:
- **Minimum Android** is the minimum android version which allowes operation for the app. The mose popular values were found: 4.1\4.0.3\6.0 and up & Varies with device.
- **Size** must exceed 1MB.
- **Currency** will be USD only.
- **Last updated** during 2020.
- Irrelevant columns filtering.


In [4]:
#Coloumn labels space fix
df.columns = [x.replace(" ", "_") for x in df.columns]

#Rows SELECT
df = df.query('(Minimum_Android == "4.1 and up" | Minimum_Android == "4.0.3 and up" | Minimum_Android == "Varies with device" | \
Minimum_Android == "6.0 and up") & not Size.str.endswith("k") & Currency == "USD" & Last_Updated.str.contains("2020")',engine='python')

#Coloumns FILTER
df = df.filter(items=['App_Name', 'Category', 'Rating', 'Rating_Count', 'Installs','Maximum_Installs', 'Price', 'Size','Released','Last_Updated','Ad_Supported', 'In_App_Purchases'])

#Early null creation
df['Size'] = df['Size'].replace(to_replace="Varies with device", #turning "Varies with device" to null.
                                   value = np.nan)
#Saving a copy of dataframe
dfcopy = df.copy(deep=True)


In [5]:
print("Dataframe reduced shape is", np.shape(df))
df.sample(10)

Dataframe reduced shape is (112092, 12)


,App_Name,Category,Rating,Rating_Count,Installs,Maximum_Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases
489279,Shiny Rose Keyboard Theme,Personalization,4.4,7.0,"1,000+",1065,0.00,67M,"Nov 29, 2018",20/02/2020 12:33,False,False
293586,Jump Rope Workout Lite,Health & Fitness,3.5,59.0,"10,000+",30195,0.00,31M,"Dec 29, 2015",25/04/2020 18:26,True,False
503027,Solitaire Tripeaks: Farm Adventure,Card,4.6,65386.0,"1,000,000+",2763217,0.00,7.3M,"Jul 29, 2019",02/07/2020 18:56,True,True
290352,jetAudio HD Music Player,Music & Audio,4.4,162021.0,"10,000,000+",14612557,0.00,31M,"May 22, 2012",18/06/2020 1:33,True,True
465378,Reverse Video: Reverse video effect & Rewind v...,Video Players & Editors,3.7,1374.0,"500,000+",613953,0.00,6.4M,"Jul 18, 2019",04/02/2020 14:29,True,False
263682,Hunting Calls HD,Sports,4.0,65.0,"10,000+",22080,0.00,18M,"Mar 25, 2015",03/04/2020 23:32,True,False
190420,English - Azerbaijani,Education,4.1,620.0,"100,000+",104995,0.00,23M,"Sep 24, 2012",20/02/2020 8:18,True,True
576895,VIRUS GAME,Casual,4.9,74.0,500+,551,0.00,53M,"Jun 5, 2020",18/06/2020 11:21,True,False
523362,Swift Dark Substratum Theme,Personalization,3.8,2384.0,"50,000+",60642,1.99,NaN,"Oct 23, 2016",20/07/2020 17:59,False,False
457389,Ratan ki Pehchan,Entertainment,0.0,0.0,"1,000+",2106,0.00,6.2M,"Jul 10, 2018",09/01/2020 20:10,True,False


# **CHECKPOINT DATA INFO**

In [6]:
print("Dataframe NULL count:")
print(df.isnull().sum(axis = 0), end="\n\n")

df.info()

Dataframe NULL count:
App_Name                1
Category                0
Rating               1880
Rating_Count         1880
Installs                0
Maximum_Installs        0
Price                   0
Size                10823
Released             6338
Last_Updated            0
Ad_Supported            0
In_App_Purchases        0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112092 entries, 20 to 603046
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App_Name          112091 non-null  object 
 1   Category          112092 non-null  object 
 2   Rating            110212 non-null  float64
 3   Rating_Count      110212 non-null  float64
 4   Installs          112092 non-null  object 
 5   Maximum_Installs  112092 non-null  int64  
 6   Price             112092 non-null  float64
 7   Size              101269 non-null  object 
 8   Released          105754 non-null  object 
 9   Last_Up

# **DATA CLEANING & OPTIMIZATION**
- A null value that was found in **App_Name** is not acceptable for further analysis, also 4 apps named: "#NAME?" should be cleaned.
- The null values in **Rating, Rating_Count, Size, and Released** columns will be replaced with mean values.
- Column dtypes will be corrected if necessary.
- Column values type will be corrected if necessary.
- Adding "Ad_bool" column, which is identical to Ad_Suppored but with 0 and 1 values, for aggregation actions such as Sum. 

In [7]:
#NULL Extraction
df = df.query('not App_Name.isnull() & not App_Name=="#NAME?"',engine='python')

#VITAL column fixes before null replacements:
#Trimming 'M' from all values in order to set as int.
df['Size'] = df['Size'].str.replace('M',"") 
df['Size'] = df['Size'].astype({'Size': 'float32'})
#Transform of "Released" coloumn to Datetime for the use of pandas.DatetimeIndex.mean
df['Released'] = pd.to_datetime(df['Released']) 

# Function replaces np.nan values in all of the data frame with relevant statistics \ dal7collab
def rep_missings(data_frame):
    d = data_frame.copy(deep = True)
    for c in data_frame:
        if data_frame[c].isnull().values.any() :
          m = data_frame[c].dropna().mean()
          d[c].fillna(m, inplace = True)
    return d;

# Activating the function
df = rep_missings(df)

In [8]:
#Adding Ad_bool column, as described above.
df['Ad_Supported'] = df['Ad_Supported'].astype({'Ad_Supported': 'string'})
df['Ad_bool']= df['Ad_Supported']
df['Ad_bool']= df['Ad_bool'].str.replace("True", "1")
df['Ad_bool']= df['Ad_bool'].str.replace("False", "0")

#Coloumn types correction
df['App_Name'] = df['App_Name'].astype({'App_Name': 'string'})
df['Category'] = df['Category'].astype({'Category': 'string'})
df['Rating'] = df['Rating'].astype({'Rating': 'float32'})
df['Rating_Count'] = df['Rating_Count'].astype({'Rating_Count': 'int64'})
df['Installs'] = df['Installs'].astype({'Installs': 'string'})
df['Price'] = df['Price'].astype({'Price': 'float32'})
df['Released'] = pd.to_datetime(df['Released']).dt.strftime('%Y-%m-%d') #Fetching only yyyy-mm-dd from the full datetime.
df['Last_Updated'] = pd.to_datetime(df['Last_Updated']).dt.strftime('%Y-%m-%d') #Fetching only yyyy-mm-dd from the full datetime.
df['Size'] = df['Size'].astype({'Size': 'float32'})
df['Ad_bool'] = df['Ad_bool'].astype({'Ad_bool': 'int8'})


print("Dataframe final shape is", np.shape(df))
df.sample(10)

Dataframe final shape is (112087, 13)


,App_Name,Category,Rating,Rating_Count,Installs,Maximum_Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases,Ad_bool
206106,Female Fitness - Lose Weight Home Workout,Health & Fitness,3.269268,25109,1+,1,0.0,18.566137,2017-10-13,2020-06-18,True,False,1
502833,Solitaire Atlantis,Card,4.200000,23206,"1,000,000+",1708835,0.0,7.300000,2014-07-24,2020-07-01,False,True,0
511622,Star 107.9,Music & Audio,4.800000,15,"1,000+",3012,0.0,7.600000,2015-01-09,2020-06-19,True,False,1
34146,日曆 2020 - 農曆,Productivity,0.000000,0,500+,519,0.0,11.000000,2020-01-13,2020-01-14,True,False,1
136536,CSE FERCO,Productivity,0.000000,0,100+,399,0.0,2.300000,2018-10-22,2020-07-20,False,False,0
86437,Block Puzzle Buddies,Puzzle,4.200000,181,"10,000+",42779,0.0,15.000000,2019-05-09,2020-02-07,True,False,1
340958,Marla Calculator,Tools,4.400000,179,"50,000+",75755,0.0,4.400000,2017-11-11,2020-03-15,True,False,1
132331,Counter ++,Tools,4.200000,8,500+,543,0.0,18.566137,2014-04-20,2020-06-01,True,False,1
548333,TRACcess eKEY®,Productivity,3.600000,97,"10,000+",44802,0.0,18.566137,2011-10-04,2020-02-26,False,False,0
208357,Find The Difference 37,Puzzle,4.700000,5128,"100,000+",347004,0.0,26.000000,2017-03-02,2020-07-22,True,True,1


# **POSTPROCESSING DATA INFO**

In [11]:
#Genereal information after preprocessing
print(df.info(),end = '\n\n')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112087 entries, 20 to 602972
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App_Name          112087 non-null  string 
 1   Category          112087 non-null  string 
 2   Rating            112087 non-null  float32
 3   Rating_Count      112087 non-null  int64  
 4   Installs          112087 non-null  string 
 5   Maximum_Installs  112087 non-null  int64  
 6   Price             112087 non-null  float32
 7   Size              112087 non-null  float32
 8   Released          112087 non-null  object 
 9   Last_Updated      112087 non-null  object 
 10  Ad_Supported      112087 non-null  string 
 11  In_App_Purchases  112087 non-null  bool   
 12  Ad_bool           112087 non-null  int8   
dtypes: bool(1), float32(3), int64(2), int8(1), object(2), string(4)
memory usage: 9.2+ MB
None



# **EXPORT TO GOOGLE DRIVE**


In [9]:
#Exporting a csv into google drive
#df.to_csv('/content/drive/My Drive/GooglePlay-Store_Optimized_v16.csv')

# **STATISTICS**

In [ ]:
#Retrieve coloumn general info 
name = 'Rating'
print(df[name].describe())
print("Coloumn's real dtype is:", df[name].dtypes)

count    112087.000000
mean          3.269279
std           1.787079
min           0.000000
25%           3.000000
50%           4.100000
75%           4.500000
max           5.000000
Name: Rating, dtype: float64
Coloumn's real dtype is: float32


In [ ]:
#Corrections for BI Robot operations
df['App_Name'] = df['App_Name'].astype({'App_Name': 'object'})
df['Category'] = df['Category'].astype({'Category': 'object'})

df.iloc[0]['Minimum_Installs'].mean()

500.0

In [ ]:
# BI Insights Robot
def bi_robot(dataframe, dimensions, measure):
    t = {}
    for d in dimensions:
        if (np.issubdtype(dataframe[d].dtype, np.number)):
            t[d] = df[d].describe()[7]    #Max value of a numeric variable
        
        else :
            t[d] = df[d].describe()[2]    #The most frequent value of any other variable

    for v in t:
        key       = v
        value     = [t[v]]
        dataframe = dataframe.loc[dataframe[key].isin(value)] 
        stat      = dataframe[measure].mean()

    print('BI Robot output:')
    print(t, end = '')  
    print(' Mean '+ measure +': ' + "{:.2f}".format(stat))
    print('---------------------') 

    #return t

In [ ]:
df_ins         = df.copy(deep = True)
filters    = ['Category']
measure    = 'Rating'
df_ins = bi_robot(df, filters, measure)
df_ins

BI Robot output:
{'Category': 'Education'} Mean Rating: 3.16
---------------------
